##CAP 4631C - Fall 2024. Assignment 5

Group 5:

Alberto Diaz

Barbara Lorenzo

Bosco Morales

Robert Rivera

##Introduction

This assignment is about Random Forest Regression and Boosted Regression Trees. **It has three questions.**

This assignment uses the *Hitters data set* from Assignment 4 for *questions 1 and 2*. Therefore, the outcome variable and the predictors are the same ones you used in Assignment 4.

It also uses the *Abalone data set for question 3.*


In [1]:
# Import useful packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, root_mean_squared_error, r2_score

from sklearn.model_selection import GridSearchCV

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Hitters dataset

In [3]:
path_hitters= "/content/drive/MyDrive/CAP4631C/Datasets_CAP4631C/Hitters_ML_HW4.csv"

In [4]:
hitters_df= pd.read_csv (path_hitters)

In [5]:
hitters_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Hits       263 non-null    int64  
 1   HmRun      263 non-null    int64  
 2   Runs       263 non-null    int64  
 3   RBI        263 non-null    int64  
 4   Years      263 non-null    int64  
 5   CAtBat     263 non-null    int64  
 6   CHits      263 non-null    int64  
 7   CHmRun     263 non-null    int64  
 8   CRuns      263 non-null    int64  
 9   CRBI       263 non-null    int64  
 10  League     263 non-null    object 
 11  Division   263 non-null    object 
 12  NewLeague  263 non-null    object 
 13  Salary     263 non-null    float64
dtypes: float64(1), int64(10), object(3)
memory usage: 28.9+ KB


Abalone dataset

In [55]:
path_abalone= "/content/drive/MyDrive/CAP4631C/Datasets_CAP4631C/abalone.csv"

In [56]:
abalone_df= pd.read_csv (path_abalone)

In [57]:
abalone_df.rename(columns={"Whole weight": "Whole_weight", "Shucked weight": "Shucked_weight","Viscera weight":"Viscera_weight", "Shell weight":"Shell_weight" }, inplace= True)

In [58]:
abalone_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             4177 non-null   object 
 1   Length          4177 non-null   float64
 2   Diameter        4177 non-null   float64
 3   Height          4177 non-null   float64
 4   Whole_weight    4177 non-null   float64
 5   Shucked_weight  4177 non-null   float64
 6   Viscera_weight  4177 non-null   float64
 7   Shell_weight    4177 non-null   float64
 8   Rings           4177 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


##Question 1 (35 points)

Apply Random Forest using the exact same method that we learned in class.

Note: If you do not apply the exact same method that we learned in class, you will be deducted 15 points out of the get-go. Notice that applying your own method (i.e., asking ChatGPT to do the homework for you :)) will imply a big penalty.

Use all the predictors when constructing the forests with the exception of “League”, “Division”, and “New League”.

In [6]:
x_all_hitters = hitters_df.iloc[:,:-4]

In [7]:
y= hitters_df['Salary']

In [8]:
x_train_hitters, x_test_hitters, y_train_hitters, y_test_hitters= train_test_split (x_all_hitters, y, test_size=0.2, random_state=1)

In [9]:
trees_hitters= np.arange(500, 1001, 100)

In [10]:
trees_hitters

array([ 500,  600,  700,  800,  900, 1000])

In [11]:
x_train_hitters.shape[1]

10

In [12]:
# Let's try 2, 4, ..., 12
# I do not try the full range to minimize computation time.

features_hitters= np.arange(2, x_train_hitters.shape[1] + 1, 2)

In [13]:
features_hitters

array([ 2,  4,  6,  8, 10])

In [14]:
hyperparam_grid_hitters = {  'n_estimators': trees_hitters,
     'max_features': features_hitters
}

In [15]:
cv_hitters = KFold (n_splits= 3, shuffle= True, random_state= 1)

In [16]:
grid_search_setting_hitters = GridSearchCV(estimator= RandomForestRegressor(random_state= 1), param_grid= hyperparam_grid_hitters, cv= cv_hitters, scoring='neg_mean_squared_error', n_jobs= -1)

In [17]:
grid_search_setting_hitters.fit(x_train_hitters, y_train_hitters)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=KFold(n_splits=3, random_state=1, shuffle=True),
             estimator=RandomForestRegressor(random_state=1), n_jobs=-1,
             param_grid={'max_features': array([ 2,  4,  6,  8, 10]),
                         'n_estimators': array([ 500,  600,  700,  800,  900, 1000])},
             scoring='neg_mean_squared_error')

In [18]:
print('Selected hyperparameter values: ', grid_search_setting_hitters.best_params_)

Selected hyperparameter values:  {'max_features': 2, 'n_estimators': 1000}


In [19]:
trees_hitters_update= np.array ([900, 950, 1000, 1050, 1100])

In [20]:
features_hitters_update= np.array ([1, 2, 3])

In [21]:
hyperparam_grid_hitters_update = {  'n_estimators': trees_hitters_update,
     'max_features': features_hitters_update
}

In [22]:
grid_search_setting_hitters_update = GridSearchCV(estimator= RandomForestRegressor(random_state= 1), param_grid= hyperparam_grid_hitters_update, cv= cv_hitters, scoring='neg_mean_squared_error', n_jobs= -1)

In [23]:
grid_search_setting_hitters_update.fit(x_train_hitters, y_train_hitters)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=KFold(n_splits=3, random_state=1, shuffle=True),
             estimator=RandomForestRegressor(random_state=1), n_jobs=-1,
             param_grid={'max_features': array([1, 2, 3]),
                         'n_estimators': array([ 900,  950, 1000, 1050, 1100])},
             scoring='neg_mean_squared_error')

In [24]:
print('Selected hyperparameter values: ', grid_search_setting_hitters_update.best_params_)

Selected hyperparameter values:  {'max_features': 2, 'n_estimators': 1050}


In [25]:
rf_hitters= RandomForestRegressor(n_estimators= 1050 , max_features= 2, random_state= 1)

In [26]:
rf_hitters.fit(x_train_hitters, y_train_hitters)

RandomForestRegressor(max_features=2, n_estimators=1050, random_state=1)

###1 a) Report the mean squared error of the chosen forest evaluated on the test dataset you created earlier (i.e., the 20% players that you left out earlier to serve as a test dataset).


In [27]:
y_pred_hitters= rf_hitters.predict (x_test_hitters)

In [28]:
mean_squared_error( y_test_hitters, y_pred_hitters)

0.16293426235904782

#### 1a Response

The mean squared error is 0.16293426235904782.

###1 b) Report the three most important predictors that form the trees part of this forest. Show how you selected the three most important predictors.

In [29]:
feature_imp_hitters_df= pd.Series(data= rf_hitters.feature_importances_, index= x_train_hitters.columns, name= "Predictor Importance in Forest")

In [30]:
feature_imp_hitters_df.sort_values(ascending= False)

,Predictor Importance in Forest
CAtBat,0.168788
CHits,0.158417
CRuns,0.152363
CRBI,0.151942
CHmRun,0.090144
Years,0.074434
Hits,0.066997
RBI,0.057117
Runs,0.049936
HmRun,0.029863


####1b  Response

From the output of the code, we observe that the top three predictors with the highest importance values are:

*   CAtBat (Importance: 0.168788)
*   CHits (Importance: 0.158417)
*   CRuns (Importance: 0.152363)

These three predictors were selected as the most important because they have the highest importance scores, indicating that they contribute the most to the decision-making process of the Random Forest model.

##Question 2 (35 points)

Obtained a boosted regression tree using the Hitters data set. Consider the following requirements when doing so:

For the learning rate, use the default value suggested by scikit-learn.
For the number of trees, try values that range from 50 to 500 in increases of 50 (that is, 50, 100, 150, …, 450, and 500).
For the max depth of the trees, try values of 2, 3, and 4.

In [31]:
# The max value here is 500, whereas it was 1000 in RFs.

trees_boosting_hitters= np.arange (50, 501, 50)

In [32]:
trees_boosting_hitters

array([ 50, 100, 150, 200, 250, 300, 350, 400, 450, 500])

In [33]:
lambda_hitters= np.array ([0.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1])

In [34]:
depth_hitters= np.array ([2, 3, 4])

In [35]:
hyperparam_grid_boosting = {
    'n_estimators': trees_boosting_hitters,
    'max_depth': depth_hitters,
    'learning_rate': lambda_hitters}

In [36]:
grid_search_setting_boost_hitters = GridSearchCV(estimator = GradientBoostingRegressor(random_state= 1), param_grid= hyperparam_grid_boosting, cv= cv_hitters, scoring='neg_mean_squared_error', n_jobs= -1)

In [37]:
grid_search_setting_boost_hitters.fit (x_train_hitters, y_train_hitters)

GridSearchCV(cv=KFold(n_splits=3, random_state=1, shuffle=True),
             estimator=GradientBoostingRegressor(random_state=1), n_jobs=-1,
             param_grid={'learning_rate': array([1.e-04, 1.e-03, 1.e-02, 5.e-02, 1.e-01, 5.e-01, 1.e+00]),
                         'max_depth': array([2, 3, 4]),
                         'n_estimators': array([ 50, 100, 150, 200, 250, 300, 350, 400, 450, 500])},
             scoring='neg_mean_squared_error')

In [38]:
print('Selected hyperparameter values: ', grid_search_setting_boost_hitters.best_params_)

Selected hyperparameter values:  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}


In [39]:
trees_boosting_hitters_updated= np.array ([50, 100, 150])

In [40]:
lambda_hitters_updated= np.array ([0.05, 0.1, 0.5])

In [41]:
depth_hitters_updated = np.array ([2, 3, 4])

In [42]:
hyperparam_grid_boosting_hitters_updated = {
    'n_estimators': trees_boosting_hitters_updated,
    'max_depth': depth_hitters_updated,
    'learning_rate': lambda_hitters_updated}

In [43]:
# n_jobs= -1

grid_search_setting_boost_hitters_updated = GridSearchCV(estimator = GradientBoostingRegressor(random_state= 1), param_grid= hyperparam_grid_boosting_hitters_updated, cv= cv_hitters, scoring='neg_mean_squared_error', n_jobs= -1)

In [44]:
grid_search_setting_boost_hitters_updated.fit (x_train_hitters, y_train_hitters)

GridSearchCV(cv=KFold(n_splits=3, random_state=1, shuffle=True),
             estimator=GradientBoostingRegressor(random_state=1), n_jobs=-1,
             param_grid={'learning_rate': array([0.05, 0.1 , 0.5 ]),
                         'max_depth': array([2, 3, 4]),
                         'n_estimators': array([ 50, 100, 150])},
             scoring='neg_mean_squared_error')

In [45]:
print('Selected hyperparameter values: ', grid_search_setting_boost_hitters_updated.best_params_)

Selected hyperparameter values:  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}


In [46]:
boosting_reg_hitters= GradientBoostingRegressor(learning_rate= 0.1, max_depth= 3, n_estimators= 50, random_state= 1)

In [47]:
boosting_reg_hitters.fit (x_train_hitters, y_train_hitters)

GradientBoostingRegressor(n_estimators=50, random_state=1)

###2 a) Report the mean squared error of the chosen solution evaluated on the test dataset you created earlier (i.e., the 20% players that you left out earlier to serve as a test dataset).

In [48]:
y_pred_boosting_hitters= boosting_reg_hitters.predict (x_test_hitters)

In [49]:
mean_squared_error( y_test_hitters, y_pred_boosting_hitters)

0.14884483277174881

#### 1a Response

The mean squared error is 0.14884483277174881.

###2 b) Use a non-naive approach to compare the performance of the random forest you obtained from question 1 with the boosted tree you obtained in 2a. Explain your comments and show your work.


###2 c) Report the three most important predictors that form the trees part of the solution. Show how you selected the three most important predictors.

Do these features coincide with the features you reported in your answer to question 1b? Explain.

In [50]:
feature_imp_boosting_hitters= pd.Series(data= boosting_reg_hitters.feature_importances_, index= x_train_hitters.columns, name= "Predictor Importance in Trees")

In [51]:
feature_imp_boosting_hitters.sort_values(ascending= False)

,Predictor Importance in Trees
CAtBat,0.469172
CHits,0.184337
Hits,0.067478
CHmRun,0.061765
CRBI,0.054981
Runs,0.045197
CRuns,0.042497
Years,0.027893
RBI,0.024633
HmRun,0.022046


Fix later: make more concise

Based on the sorted values, the three most important predictors in the Gradient Boosting model are:


*   CAtBat (Importance: 0.469172)
*   CHits (Importance: 0.184337)
*   Hits (Importance: 0.067478)


**Comparison with 1b:**


In Question 1b, we performed feature importance analysis using a Random Forest model. The top three predictors identified were:

*   CAtBat (Importance: 0.168788)
*   CHits (Importance: 0.158417)
*   CRuns (Importance: 0.152363)

When comparing the two models (Random Forest vs. Gradient Boosting), the first two predictors, CAtBat and CHits, remain consistently the most important in both models. However, the third predictor differs: CRuns was the third most important in Random Forest, while Hits emerged as the third most important in the Gradient Boosting model.

This difference may arise due to the way the two algorithms function. Random Forest uses bagging and constructs trees independently, while Gradient Boosting builds trees sequentially with a focus on correcting the errors of previous trees. Therefore, the models weigh feature contributions differently, leading to some variation in predictor importance.








##Question 3 (30 points)

Apply Random Forest using ALMOST the exact same method that we learned in class. The only modification you need to make is to consider a range of values for the ‘number of features’ that extends from 1 to the total number of predictors in the data set.

Note: If you do not apply the exact same method that we learned in class, you will be deducted 15 points out of the get-go.

The outcome variable to predict is ‘Rings’ (the last column), which records the number of rings in the abalone (the number of rings, in turn, is related to the abalone’s age)

Do NOT include the first column, ‘Sex’, as a predictor in your answer.

Before starting to work, I suggest you change the name of some of the columns in the dataset. To do so, you should run the following code in a Colab cell:

`abalone_df.rename(columns={"Whole weight": "Whole_weight", "Shucked weight": "Shucked_weight","Viscera weight":"Viscera_weight", "Shell weight":"Shell_weight" }, inplace= True)`

In [63]:
x_all_abalone = abalone_df.iloc[:,1:-1]

In [64]:
x_all_abalone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Length          4177 non-null   float64
 1   Diameter        4177 non-null   float64
 2   Height          4177 non-null   float64
 3   Whole_weight    4177 non-null   float64
 4   Shucked_weight  4177 non-null   float64
 5   Viscera_weight  4177 non-null   float64
 6   Shell_weight    4177 non-null   float64
dtypes: float64(7)
memory usage: 228.6 KB


In [65]:
y= abalone_df['Rings']

In [66]:
x_train_abalone, x_test_abalone, y_train_abalone, y_test_abalone= train_test_split (x_all_abalone, y, test_size=0.2, random_state=1)

In [67]:
trees_abalone= np.arange(500, 1001, 100)

In [70]:
trees_abalone

array([ 500,  600,  700,  800,  900, 1000])

In [71]:
x_train_abalone.shape[1]

7

In [86]:
features_abalone= np.arange(1, x_train_abalone.shape[1] + 1, 1)

In [87]:
features_abalone

array([1, 2, 3, 4, 5, 6, 7])

In [74]:
hyperparam_grid_abalone = {  'n_estimators': trees_abalone,
     'max_features': features_abalone
}

In [85]:
cv_abalone = KFold (n_splits= 3, shuffle= True, random_state= 1)

In [90]:
grid_search_setting_abalone = GridSearchCV(estimator= RandomForestRegressor(random_state= 1), param_grid= hyperparam_grid_abalone, cv= cv_abalone, scoring='neg_mean_squared_error', n_jobs= -1)

In [92]:
grid_search_setting_abalone.fit(x_train_abalone, y_train_abalone)

GridSearchCV(cv=KFold(n_splits=3, random_state=1, shuffle=True),
             estimator=RandomForestRegressor(random_state=1), n_jobs=-1,
             param_grid={'max_features': array([1, 3, 5, 7, 9]),
                         'n_estimators': array([ 500,  600,  700,  800,  900, 1000])},
             scoring='neg_mean_squared_error')

In [ ]:
print('Selected hyperparameter values: ', grid_search_setting_abalone.best_params_)

In [ ]:
trees_abalone_update= np.array ([900, 950, 1000, 1050, 1100])

In [ ]:
features_abalone_update= np.array ([1, 2, 3])

In [ ]:
hyperparam_grid_abalone_update = {  'n_estimators': trees_abalone_update,
     'max_features': features_abalone_update
}

In [ ]:
grid_search_setting_abalone_update = GridSearchCV(estimator= RandomForestRegressor(random_state= 1), param_grid= hyperparam_grid_abalone_update, cv= cv_abalone, scoring='neg_mean_squared_error', n_jobs= -1)

In [ ]:
grid_search_setting_abalone_update.fit(x_train_abalone, y_train_abalone)

###3 a) Report the mean squared error of the chosen forest evaluated on the test dataset you created earlier.

How would you judge the value of this error? Low (good) enough? Justify.